IMDB Movie Comments Sentiment Analysis Practice
===
Kenny Hsieh, 2018/3/15

- Construct MLP, RNN, LSTM Model to overcome the task

## Download the IMDB Dataset

In [0]:
import urllib.request
import os
import tarfile

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath = "daclImdb_v1.tar.gz"
if not os.path.isfile(filepath): 
  response = urllib.request.urlretrieve(url, filepath)
  print("Downloaded : ", response)
  
tfile = tarfile.open("daclImdb_v1.tar.gz", 'r:gz')
result = tfile.extractall()

In [12]:
!ls -ls

total 82164
    4 drwxr-xr-x 4 7297 1000     4096 Jun 26  2011 aclImdb
82156 -rw-r--r-- 1 root root 84125825 Mar 15 08:06 daclImdb_v1.tar.gz
    4 drwxr-xr-x 1 root root     4096 Mar 15 08:02 datalab


## Data Preprocessing : Remove the html Tag

In [0]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import re
def rm_tags(text):
  re_tag = re.compile(r'<[^>]+>')
  return re_tag.sub("", text)

import os
def read_files(filetype):
  path = "aclImdb/"
  file_list = []
  
  positive_path = path + filetype + "/pos/"
  for f in os.listdir(positive_path):
    file_list += [positive_path + f]
    
  negative_path = path + filetype + "/neg/"
  for f in os.listdir(negative_path):
    file_list += [negative_path + f]
    
  print("Read", filetype, 'files :', len(file_list))
  
  all_labels = ([1] * 12500 + [0] * 12500)
  all_texts = []
  for fi in file_list:
    with open(fi, encoding = 'utf-8') as file_input:
      all_texts += [rm_tags("".join(file_input.readlines()))]
      
  return all_labels, all_texts

In [16]:
y_train, train_text = read_files("train")
y_test, test_text = read_files("test")

print("Label : ", y_train[0])
print("Context : ", train_text[0])

Read train files : 25000
Read test files : 25000


## Construct the Tokenizer from Training Data

In [59]:
token = Tokenizer(num_words = 4000)
token.fit_on_texts(train_text)

print(token.document_count)
print(token.word_index)

25000
{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'in': 7, 'it': 8, 'i': 9, 'this': 10, 'that': 11, 'was': 12, 'as': 13, 'for': 14, 'with': 15, 'movie': 16, 'but': 17, 'film': 18, 'on': 19, 'not': 20, 'you': 21, 'are': 22, 'his': 23, 'have': 24, 'be': 25, 'he': 26, 'one': 27, 'all': 28, 'at': 29, 'by': 30, 'an': 31, 'they': 32, 'who': 33, 'so': 34, 'from': 35, 'like': 36, 'her': 37, 'or': 38, 'just': 39, 'about': 40, "it's": 41, 'out': 42, 'has': 43, 'if': 44, 'some': 45, 'there': 46, 'what': 47, 'good': 48, 'more': 49, 'when': 50, 'very': 51, 'up': 52, 'no': 53, 'time': 54, 'she': 55, 'even': 56, 'my': 57, 'would': 58, 'which': 59, 'only': 60, 'story': 61, 'really': 62, 'see': 63, 'their': 64, 'had': 65, 'can': 66, 'were': 67, 'me': 68, 'well': 69, 'than': 70, 'we': 71, 'much': 72, 'been': 73, 'get': 74, 'bad': 75, 'will': 76, 'also': 77, 'do': 78, 'into': 79, 'people': 80, 'other': 81, 'first': 82, 'great': 83, 'because': 84, 'how': 85, 'him': 86, 'most': 87, "don't": 88,

## Tokenlize Text to Sequence

In [26]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

print(train_text[0])
print(x_train_seq[0])

Maria Braun is an extraordinary woman presented fully and very credibly, despite being so obtuse as to border on implausibility. She will do everything to make her marriage work, including shameless opportunism and sexual manipulation. And thus beneath the vicey exterior, she reveals a rather sweet value system. The film suffers from an abrupt and unexpected ending which afterwards feels wholly inadequate, with the convenience familiar from ending your school creative writing exercise with 'and then I woke up'. It is also book-ended at the other end with the most eccentric title sequence I've ever seen, but don't let any of that put you off.
[6, 31, 251, 1346, 1310, 2, 51, 462, 108, 34, 13, 5, 19, 55, 76, 78, 282, 5, 93, 37, 1338, 153, 582, 2, 860, 2, 1339, 1, 55, 3, 243, 1041, 1103, 1502, 1, 18, 35, 31, 2, 273, 59, 759, 15, 1, 1074, 35, 273, 125, 391, 1514, 483, 15, 91, 9, 8, 6, 77, 270, 1050, 29, 1, 81, 126, 15, 1, 87, 421, 716, 203, 122, 106, 17, 88, 383, 97, 4, 11, 272, 21, 121]


## Padding Sequence to Same Length

In [60]:
x_train = sequence.pad_sequences(x_train_seq, maxlen = 400)
x_test = sequence.pad_sequences(x_test_seq, maxlen = 400)

print("Before Padding : ", len(x_train_seq[0]))
print("After Padding : ", len(x_train[0]))

# Forward add 0 / delete exceed numeric 

Before Padding :  80
After Padding :  400


## Multilayer Perceptron (MLP)

In [61]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding # Numeric list to Vector list

model = Sequential()
model.add(Embedding(output_dim = 32,
                   input_dim = 4000,
                   input_length = 400))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(units = 256,
               activation = 'relu'))
model.add(Dropout(0.35))

model.add(Dense(units = 1,
               activation = 'sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 400, 32)           128000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 400, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               3277056   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 3,405,313
Trainable params: 3,405,313
Non-trainable params: 0
_________________________________________________________________


In [62]:
model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

train_history = model.fit(x_train, y_train,
                          validation_split = 0.2,
                         batch_size = 100,
                         epochs = 10)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 3s 151us/step - loss: 0.4974 - acc: 0.7452 - val_loss: 0.4132 - val_acc: 0.8174
Epoch 2/10
20000/20000 [==============================] - 3s 130us/step - loss: 0.2397 - acc: 0.9058 - val_loss: 0.5054 - val_acc: 0.7830
Epoch 3/10
20000/20000 [==============================] - 3s 129us/step - loss: 0.1274 - acc: 0.9552 - val_loss: 0.9150 - val_acc: 0.6814
Epoch 4/10
20000/20000 [==============================] - 3s 131us/step - loss: 0.0569 - acc: 0.9827 - val_loss: 0.8144 - val_acc: 0.7620
Epoch 5/10
20000/20000 [==============================] - 3s 130us/step - loss: 0.0298 - acc: 0.9915 - val_loss: 0.8951 - val_acc: 0.7728
Epoch 6/10
 3900/20000 [====>.........................] - ETA: 1s - loss: 0.0132 - acc: 0.9974

20000/20000 [==============================] - 3s 129us/step - loss: 0.0171 - acc: 0.9956 - val_loss: 0.9935 - val_acc: 0.7724
Epoch 7/10
20000/20000 [==============================] - 3s 131us/step - loss: 0.0144 - acc: 0.9954 - val_loss: 0.9993 - val_acc: 0.7900
Epoch 8/10
20000/20000 [==============================] - 3s 130us/step - loss: 0.0120 - acc: 0.9961 - val_loss: 1.0770 - val_acc: 0.7880
Epoch 9/10
20000/20000 [==============================] - 3s 131us/step - loss: 0.0161 - acc: 0.9946 - val_loss: 1.4964 - val_acc: 0.7324
Epoch 10/10
20000/20000 [==============================] - 3s 130us/step - loss: 0.0167 - acc: 0.9942 - val_loss: 0.8950 - val_acc: 0.8310


In [63]:
scores = model.evaluate(x_test, y_test, verbose = 1)
scores[1]

25000/25000 [==============================] - 2s 98us/step


0.84728

In [64]:
predict = model.predict_classes(x_test)
predict_classes = predict.reshape(-1)
predict_classes[:10]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

## Observe the Specific Commits with Actual & Predict Label

In [65]:
Sentiment_dict = {1 : 'Pos', 0 : 'Neg'}
def display_test_sentiment(i):
  print(test_text[i])
  print('Actual Answer : ', Sentiment_dict[y_test[i]])
  print('Predict Answer : ', Sentiment_dict[predict_classes[i]])
  
display_test_sentiment(2)

I've watched this movie on a fairly regular basis for most of my life, and it never gets old. For all the snide remarks and insults (mostly from David Spade), "Tommy Boy" has a giant heart. And that's what keeps this movie funny after all these years.Tommy Callahan (Chris Farley) is the son of Big Tom Callahan (Brian Dennehy), master car parts salesman, and has ridden on that all his life. But after his died dies on his wedding day, Tommy learns that the company is in debt, and about to be bought by Ray Zalinsky (Dan Akroyd), the owner of a huge car parts company. So in order to save the company, Tommy has to go on the road to sell the company's new brake pads. Along for the ride, though not by choice, is Richard Hayden (David Spade) a former classmate of Tommy's who was Big Tom's right-hand man.The movie rides on the chemistry between the two SNL stars (and real-life best friends) Chris Farley and David Spade. The duo has enough comic energy going between them to power the world. It's

## Check User Prompt Whether is Pos/Neg

In [0]:
def prompt_predict(input_text):
  token_text = token.texts_to_sequences([input_text])
  pad_text = sequence.pad_sequences(token_text, maxlen = 400)
  predict_result = model.predict_classes(pad_text)
  
  print("This review sentiment is : " + Sentiment_dict[predict_result[0][0]])

In [72]:
user_input = '''Featuring the very latest in computer generated effects and a plot that would insult a retarded six year old, Ironman is the latest in a long line of Hollywood product culled from the dizzying literary heights of superhero comic books.

Product this disposable doesn't happen by accident. Teams of industry product makers work around the clock for years to make something this tasteless, tedious and utterly bland. Don't even try to characterize this product as a "movie." It simply isn't.

If Ironman was edible, it would be a jar of baby food. Library paste flavored baby food. If it were music, it would be chopsticks. If it were a vehicle, it would be a tricycle. A tricycle with training wheels. If it were...

What? Oh yeah, the product. It's something about the usual gazillionaire playboy who flies around in a sooper-dooper extra-gadgety metal suit and frees Afghanistan from the Evil Brown People, making the country safe for, I dunno, oil companies I guess. The plot is such a pile of moronic sub-juvenile drivel it's impossible to focus on for more than a few minutes before being distracted by something more interesting like, say, the butter-flavored grease stains on the bottom of your popcorn bag.

Contempt for it's audience drips from every frame of this product. No doubt the droolcase fanboy target audience will be thrilled to down to their Batman Under-Roos by Downey's lackluster performance as he sleepwalks through the product's repulsive hero role. The product is certainly simple enough for people who regard Cheetos as food group and think repetitively boring video games are the apex of computer technology.

But aside from it's pimply focus group, the only other audiences to which this product could possibly appeal would be coma patients and Scientologists. And you'd probably have to tie down the coma patients.'''

prompt_predict(user_input)

This review sentiment is : Neg


## Recurrent Neural Network (RNN)

In [74]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

model = Sequential()
model.add(Embedding(output_dim = 32,
                   input_dim = 4000,
                   input_length = 400))
model.add(Dropout(0.2))

model.add(SimpleRNN(units = 16))
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.35))

model.add(Dense(units = 1, activation = 'sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 400, 32)           128000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 400, 32)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_5 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 133,393
Trainable params: 133,393
Non-trainable params: 0
_________________________________________________________________
None

In [76]:
model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

train_history = model.fit(x_train, y_train,
                          validation_split = 0.2,
                         batch_size = 100,
                         epochs = 10)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 37s 2ms/step - loss: 0.4983 - acc: 0.7511 - val_loss: 0.7517 - val_acc: 0.6516
Epoch 2/10
20000/20000 [==============================] - 37s 2ms/step - loss: 0.3681 - acc: 0.8476 - val_loss: 0.5727 - val_acc: 0.7232
Epoch 3/10
 3500/20000 [====>.........................] - ETA: 29s - loss: 0.3216 - acc: 0.8700

20000/20000 [==============================] - 37s 2ms/step - loss: 0.3175 - acc: 0.8702 - val_loss: 0.5894 - val_acc: 0.7352
Epoch 4/10
20000/20000 [==============================] - 37s 2ms/step - loss: 0.2851 - acc: 0.8866 - val_loss: 0.5959 - val_acc: 0.7454
Epoch 5/10
11600/20000 [================>.............] - ETA: 14s - loss: 0.2352 - acc: 0.9093

20000/20000 [==============================] - 37s 2ms/step - loss: 0.2499 - acc: 0.9032 - val_loss: 0.5526 - val_acc: 0.7688
Epoch 6/10
20000/20000 [==============================] - 38s 2ms/step - loss: 0.2228 - acc: 0.9156 - val_loss: 0.6373 - val_acc: 0.7586
Epoch 7/10
13500/20000 [===================>..........] - ETA: 11s - loss: 0.1826 - acc: 0.9293

20000/20000 [==============================] - 37s 2ms/step - loss: 0.1882 - acc: 0.9262 - val_loss: 0.6198 - val_acc: 0.7376
Epoch 8/10
20000/20000 [==============================] - 37s 2ms/step - loss: 0.1648 - acc: 0.9395 - val_loss: 0.7631 - val_acc: 0.7608
Epoch 9/10
14000/20000 [====================>.........] - ETA: 10s - loss: 0.1234 - acc: 0.9561

20000/20000 [==============================] - 38s 2ms/step - loss: 0.1308 - acc: 0.9524 - val_loss: 0.6949 - val_acc: 0.7846
Epoch 10/10
20000/20000 [==============================] - 38s 2ms/step - loss: 0.1151 - acc: 0.9576 - val_loss: 0.8652 - val_acc: 0.7478


In [77]:
scores = model.evaluate(x_test, y_test, verbose = 1)
scores[1]

25000/25000 [==============================] - 27s 1ms/step


0.80488

## Long Short-term Memory (LSTM)

In [81]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

model = Sequential()
model.add(Embedding(output_dim = 32,
                   input_dim = 4000,
                   input_length = 400))
model.add(Dropout(0.2))

model.add(LSTM(units = 32))
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.35))

model.add(Dense(units = 1, activation = 'sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 400, 32)           128000    
_________________________________________________________________
dropout_9 (Dropout)          (None, 400, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_9 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 145,025
Trainable params: 145,025
Non-trainable params: 0
_________________________________________________________________
None

In [82]:
model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

train_history = model.fit(x_train, y_train,
                          validation_split = 0.2,
                          batch_size = 100,
                          epochs = 10)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 136s 7ms/step - loss: 0.4862 - acc: 0.7573 - val_loss: 0.3778 - val_acc: 0.8292
Epoch 2/10
20000/20000 [==============================] - 135s 7ms/step - loss: 0.2990 - acc: 0.8792 - val_loss: 0.4966 - val_acc: 0.7990
Epoch 3/10
 3300/20000 [===>..........................] - ETA: 1:47 - loss: 0.2771 - acc: 0.8903

20000/20000 [==============================] - 135s 7ms/step - loss: 0.2626 - acc: 0.8958 - val_loss: 0.2943 - val_acc: 0.8748
Epoch 4/10
20000/20000 [==============================] - 134s 7ms/step - loss: 0.2457 - acc: 0.9041 - val_loss: 0.3320 - val_acc: 0.8552
Epoch 5/10
11300/20000 [===============>..............] - ETA: 55s - loss: 0.2290 - acc: 0.9109

20000/20000 [==============================] - 134s 7ms/step - loss: 0.2324 - acc: 0.9094 - val_loss: 0.3982 - val_acc: 0.8400
Epoch 6/10
20000/20000 [==============================] - 134s 7ms/step - loss: 0.2185 - acc: 0.9147 - val_loss: 0.3327 - val_acc: 0.8598
Epoch 7/10
13200/20000 [==================>...........] - ETA: 43s - loss: 0.2048 - acc: 0.9218

20000/20000 [==============================] - 135s 7ms/step - loss: 0.2100 - acc: 0.9188 - val_loss: 0.6115 - val_acc: 0.7852
Epoch 8/10
20000/20000 [==============================] - 133s 7ms/step - loss: 0.1865 - acc: 0.9289 - val_loss: 0.4777 - val_acc: 0.8062
Epoch 9/10
13600/20000 [===================>..........] - ETA: 40s - loss: 0.1739 - acc: 0.9339

20000/20000 [==============================] - 134s 7ms/step - loss: 0.1843 - acc: 0.9301 - val_loss: 0.5181 - val_acc: 0.7830
Epoch 10/10
20000/20000 [==============================] - 133s 7ms/step - loss: 0.1704 - acc: 0.9353 - val_loss: 0.6724 - val_acc: 0.7642


In [0]:
scores = model.evaluate(x_test, y_test, verbose = 1)
scores[1]

15584/25000 [=================>............] - ETA: 38s